In [163]:
# To run this file correctly, do the following
# indent : entire code inside to function
# delete : For Dev psycopg2 and dotnet env
# uncomment: For Production running on Python Enviroment
# uncomment : return and dtNow=datetime.now()
# remove and  ae.id in (22,26)
# uncomment email_info for sent mail
# uncomment delete file

In [164]:
# def notify_imcomplete_pm_to_team():

In [165]:
import pandas as pd
from dotenv import dotenv_values

from datetime import datetime,date
from dateutil import relativedelta
import os


# Paramter & Config values

In [166]:
temp_doc="temp_pm_notifcation"
cutOffPMDate="2024-01-01"

# Retrive data from SMartDB Postgresql


In [167]:
# For Production running on Python Enviroment
# from django.db import connection
# def get_postgres_conn():
#   return connection


# For Dev
env_path=r'D:\PythonDev\Yit\SMartApp_ScriptDev\.env'
config = dotenv_values(dotenv_path=env_path)

import psycopg2
import psycopg2.extras as extras
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error


In [168]:
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

# Get the next month 
* Set Window Sheduler to monthly run at the fist dsate of The month
* No matter what day you run this job , the program will  get only all PMs over the current month

In [169]:
#dtNow= datetime.strptime(datetime(2023,12,1,6,0).strftime('%Y-%m-%d'),'%Y-%m-%d')
dtNow=datetime.now()

dt = datetime.strptime(dtNow.strftime('%Y-%m-%d'),'%Y-%m-%d')
print(dt)

first_day_month = datetime(dt.year,dt.month,1)
first_day_next_month= dt + relativedelta.relativedelta(months=1, day=1)
print(first_day_month )   
print(first_day_next_month ) 


2023-11-30 00:00:00
2023-11-01 00:00:00
2023-12-01 00:00:00


# Retrive  and Transaform data

In [170]:
def do_something_df(df):
    df = df.fillna(value='')
    df=df.dropna(subset=['email_teamlead'])
    # df = df.reset_index(drop=False)
    # df.insert(0, "No", df["index"]+1,True)
    # df=df.drop(columns=["index"])
    return df

In [171]:
sql_item = f"""
 select  ac.company_full_name as "ชื่อลูกค้า",
 ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" , ap.project_name as "ชื่อโครงการ",

             ai.serial_number as "Serial",
             (select  productype_name from app_product_type where id=ai.product_type_id ) as "ProudctType",
             (select  brand_name from app_brand where id=ai.brand_id ) as "Brand",
              (select  model_name from app_model where id=ai.model_id ) as "Model",

             TO_CHAR(pm.planned_date,'DD Mon YYYY') as "แผนจะทำPM",TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
             pm.remark as  "งวดPM",
             ae.employee_name as "หัวหน้าทีม",
             (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Planed Engineer",

           (select employee_name from app_employee eng_pm  where eng_pm.id=pm_item.pm_engineer_id ) as "Operation Engineer",
           TO_CHAR(pm_item.actual_date,'DD Mon YYYY') as "ActualDate",

            (select employee_name from app_employee eng_doc  where eng_doc.id=pm_item.document_engineer_id ) as "Doc Engineer",
           TO_CHAR(pm_item.document_date,'DD Mon YYYY') as "DocumentDate",
           
           pm_item.call_number as "Call Number",pm_item.pm_document_number as "Doc Number",
           pm_item.remark as "Remark"
           
           ,ae.employee_name as "หัวหน้าทีม" 
           ,ae.employee_email as "email_teamlead"

from app_pm_inventory as pm_item
left join app_inventory ai on ai.id = pm_item.inventory_id
-- inner join app_product_type  product_type on ai.product_type_id = product_type.id
left join app_preventivemaintenance pm on pm.id = pm_item.pm_master_id
left join app_project ap on ap.id = pm.project_id
left join app_company ac on ac.id = ap.company_id
left join app_employee ae on ae.id =pm.team_lead_id

where pm_item.is_pm=True 
and  
(pm.planned_date>='{first_day_month}' and pm.planned_date<'{first_day_next_month}'  )

and  ( pm_item.actual_date is null or pm_item.document_date is null 
       or pm_item.pm_engineer_id is null or  pm_item.document_engineer_id is null  )                                                       

and  ae.id in (22,26)

order by  ac.company_full_name,ap.enq_id,pm.remark

    """

# to cover incomplete inventoru , you need to determine cutoff date to check pm item.
# (pm.planned_date>='{cutOffPMDat}' and pm.planned_date<'{first_day_next_month}'  )

#  pongthorn=trong and chatchawan-seng and  
# ae.id in (22,26)

# print(sql_item)

In [172]:
dfItem=list_data(sql_item,None,get_postgres_conn())
dfItem=do_something_df(dfItem)
dfItem.info()
dfItem.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ชื่อลูกค้า          9 non-null      object
 1   เลขที่สัญญา         9 non-null      object
 2   ENQ                 9 non-null      object
 3   ชื่อโครงการ         9 non-null      object
 4   Serial              9 non-null      object
 5   ProudctType         9 non-null      object
 6   Brand               9 non-null      object
 7   Model               9 non-null      object
 8   แผนจะทำPM           9 non-null      object
 9   วันสุดท้ายที่ทำPM   9 non-null      object
 10  งวดPM               9 non-null      object
 11  หัวหน้าทีม          9 non-null      object
 12  Planed Engineer     9 non-null      object
 13  Operation Engineer  9 non-null      object
 14  ActualDate          9 non-null      object
 15  Doc Engineer        9 non-null      object
 16  DocumentDate        9 non-null

,ชื่อลูกค้า,เลขที่สัญญา,ENQ,ชื่อโครงการ,Serial,ProudctType,Brand,Model,แผนจะทำPM,วันสุดท้ายที่ทำPM,...,หัวหน้าทีม,Planed Engineer,Operation Engineer,ActualDate,Doc Engineer,DocumentDate,Call Number,Doc Number,Remark,email_teamlead
0,Yit In Tsoi,Test112,ENQ/65-0001PMDoc,Test-PMDoc,-,Software,Microsoft,Mail Exchange,01 Nov 2023,30 Nov 2023,...,Pongthorn Sangkaphet,Pongthorn Sangkaphet,,,,,,,,Pongthorn.Sa@yipintsoi.com
1,Yit In Tsoi,Test112,ENQ/65-0001PMDoc,Test-PMDoc,-,Software,Microsoft,POWER BI,01 Nov 2023,30 Nov 2023,...,Pongthorn Sangkaphet,Pongthorn Sangkaphet,,,,,,,,Pongthorn.Sa@yipintsoi.com
2,Yit In Tsoi,Test112,ENQ/65-0001PMDoc,Test-PMDoc,-,Software,Microsoft,POWER BI,01 Nov 2023,30 Nov 2023,...,Pongthorn Sangkaphet,,Attasit somsuk,30 Nov 2023,Boonlert Kawta,,,,ok,Pongthorn.Sa@yipintsoi.com
3,Yit In Tsoi,Test112,ENQ/65-0001PMDoc,Test-PMDoc,-,Software,YIP,Purge System,15 Nov 2023,30 Nov 2023,...,Chatchawan Wongviriyasit,Pongthorn Sangkaphet,,,,,,,ok all done,chatchawan.wo@yipintsoi.com
4,Yit In Tsoi,Test112,ENQ/65-0001PMDoc,Test-PMDoc,bc001,Software,YIP,Yit-Blockchain,15 Nov 2023,30 Nov 2023,...,Chatchawan Wongviriyasit,Pongthorn Sangkaphet,,,,,,,ok all done,chatchawan.wo@yipintsoi.com


# Gen excel and send mail for each team lead

In [173]:
emailList=dfItem["email_teamlead"].unique().tolist()
for email in  emailList:
    
    name=email.split("@")[0]
    name=name.replace(".","_")
    
    file_name=f"{name}_IncompletPM_{first_day_month.strftime('%b%y')}_{dtNow.strftime('%d%m%y%H%M')}.xlsx" 
    file_path=f"{temp_doc}/{file_name}"
    
    dfByTeamLead=dfItem.query("email_teamlead==@email")
    dfByTeamLead=dfByTeamLead.drop(columns=["email_teamlead"])
    
    with pd.ExcelWriter(file_path) as writer:
        dfByTeamLead.to_excel(writer, sheet_name="PM-Item",index=False)
        print(f"Exported {file_name} file for email successfully.")
        
    is_sussessful=False    
    # # Email Office 365
    title = f'SmartPM: Incomplete-PM To TeamLead - {file_name}'
    
    content = f'<h3>Download  Incomplete-PM  excel file.</h3>'
    content = f'{content}<h4>In each row as attached file, some of these columns have not been filled in data.</h4>'
    content = f'{content}<h5>Operation Engineer,ActualDate,Doc Engineer,DocumentDate.</h5>'
    print(content)
    
    listRecipients =[email]
    print(f"It is about to send email to {listRecipients}")
    
    # email_info = {'subject': title, 'message': content, 'send_to':listRecipients}
    # is_sussessful = send_email_with_excel_file(email_info,file_path,file_name)
    # print("Sent mail successfully.")
    
    # os.remove(f"{file_path}")
    # print(f"Deleted file {file_path} for email attachemnt  succesfully.")
    

Exported Pongthorn_Sa_IncompletPM_Nov23_3011232112.xlsx file for email successfully.
<h3>Download  Incomplete-PM  excel file.</h3><h4>In each row as attached file, some of these columns have not been filled in data.</h4><h5>Operation Engineer,ActualDate,Doc Engineer,DocumentDate.</h5>
It is about to send email to ['Pongthorn.Sa@yipintsoi.com']
Exported chatchawan_wo_IncompletPM_Nov23_3011232112.xlsx file for email successfully.
<h3>Download  Incomplete-PM  excel file.</h3><h4>In each row as attached file, some of these columns have not been filled in data.</h4><h5>Operation Engineer,ActualDate,Doc Engineer,DocumentDate.</h5>
It is about to send email to ['chatchawan.wo@yipintsoi.com']


In [ ]:
# return is_sussessful